# **Colab From https://github.com/TheLastBen/fast-stable-diffusion, if you have any issues, feel free to discuss them.**









In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#@markdown # Installing hlky repo
%%capture
%cd /content/gdrive/MyDrive
!git clone https://github.com/sd-webui/stable-diffusion-webui
%cd /content/gdrive/MyDrive/stable-diffusion-webui/
!mkdir -p cache/{huggingface,torch}
%cd /content/
!ln -s /content/gdrive/MyDrive/stable-diffusion-webui/cache/huggingface ../root/.cache/
!ln -s /content/gdrive/MyDrive/stable-diffusion-webui/cache/torch ../root/.cache/

In [3]:
#@markdown # Download model files

!apt install megatools
!megadl "https://mega.co.nz/#!FxclSKoL!EFSM4nLlXMuOvBLkoZNmtOH4Y8oycjrU7h2Hn6mKl1k"


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  megatools
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 148 kB of archives.
After this operation, 1,097 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 megatools amd64 1.9.98-1build2 [148 kB]
Fetched 148 kB in 1s (260 kB/s)
Selecting previously unselected package megatools.
(Reading database ... 159447 files and directories currently installed.)
Preparing to unpack .../megatools_1.9.98-1build2_amd64.deb ...
Unpacking megatools (1.9.98-1build2) ...
Setting up megatools (1.9.98-1build2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Downloaded u1000.ckpt
mv: cannot stat '/content/stable-diffusion-webui/u1000.ckpt': No s

In [4]:
#@markdown # Installing Requirements
%%capture
import os
if not os.path.exists('/content/gdrive/MyDrive/stable-diffusion-webui/src/taming-transformers/taming'):
  %cd /content/gdrive/MyDrive/stable-diffusion-webui/
  !pip install -e git+https://github.com/CompVis/taming-transformers#egg=taming-transformers
  !pip install -e git+https://github.com/openai/CLIP#egg=clip
  !pip install -e git+https://github.com/TencentARC/GFPGAN#egg=GFPGAN
  !pip install -e git+https://github.com/xinntao/Real-ESRGAN#egg=realesrgan
  !pip install -e git+https://github.com/hlky/k-diffusion-sd#egg=k_diffusion
  !pip install -e git+https://github.com/devilismyfriend/latent-diffusion#egg=latent-diffusion
!pip install git+https://github.com/openai/CLIP#egg=clip
!pip install scikit-image
!pip install clean-fid
!pip install accelerate
!pip install einops
!pip install jsonmerge
!pip install wandb
!pip install resize_right
!pip install torchdiffeq
!pip install numpy==1.21.6
!pip install albumentations==0.4.3
!pip install diffusers==0.3.0
!pip install facexlib>=0.2.3
!pip install gradio==3.1.6
!pip install imageio-ffmpeg==0.4.2
!pip install imageio==2.9.0
!pip install kornia==0.6
!pip install omegaconf==2.1.1
!pip install opencv-python-headless==4.6.0.66
!pip install piexif==1.1.3
!pip install pudb==2019.2
!pip install pynvml==11.4.1
!pip install python-slugify>=6.1.2
!pip install pytorch-lightning
!pip install torch-fidelity==0.3.0
!pip install transformers==4.19.2
!pip install triton==2.0.0.dev20220701

In [5]:
#@markdown # GFGAN + ESRGAN + LDSR models download
%%capture
import os
%cd /content/gdrive/MyDrive/stable-diffusion-webui/src/gfpgan/
!pip install basicsr facexlib yapf lmdb opencv-python pyyaml tb-nightly --no-deps
!python setup.py develop
!pip install realesrgan
if not os.path.exists('/content/gdrive/MyDrive/stable-diffusion-webui/src/gfpgan/experiments/pretrained_models/GFPGANv1.3.pth'):
  !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

%cd /content/gdrive/MyDrive/stable-diffusion-webui/src/realesrgan/

if not os.path.exists('/content/gdrive/MyDrive/stable-diffusion-webui/src/realesrgan/experiments/pretrained_models/RealESRGAN_x4plus.pth'):
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models

if not os.path.exists('/content/gdrive/MyDrive/stable-diffusion-webui/src/latent-diffusion/experiments/pretrained_models/model.ckpt'):
  %cd /content/gdrive/MyDrive/stable-diffusion-webui/src
  %cd latent-diffusion
  %mkdir -p experiments/
  %cd experiments/
  %mkdir -p pretrained_models
  %cd pretrained_models
  !wget -O project.yaml https://heibox.uni-heidelberg.de/f/31a76b13ea27482981b4/?dl=1
  !wget -O model.ckpt https://heibox.uni-heidelberg.de/f/578df07c8fc04ffbadf3/?dl=1

In [6]:
#@markdown # Installing xformers
%%capture
import os
from IPython.display import HTML
from subprocess import getoutput
if not os.path.exists('/content/gdrive/MyDrive/stable-diffusion-webui/scripts/xformers'):
  %cd /content/gdrive/MyDrive/stable-diffusion-webui/src
  !git clone https://github.com/facebookresearch/xformers
  !cp -R '/content/gdrive/MyDrive/stable-diffusion-webui/frontend' '/content/gdrive/MyDrive/stable-diffusion-webui/scripts'
  !cp -R '/content/gdrive/MyDrive/stable-diffusion-webui/src/taming-transformers/taming' '/content/gdrive/MyDrive/stable-diffusion-webui/scripts'
  !cp -R '/content/gdrive/MyDrive/stable-diffusion-webui/ldm' '/content/gdrive/MyDrive/stable-diffusion-webui/scripts/ldm'
  !cp -R '/content/gdrive/MyDrive/stable-diffusion-webui/src/k-diffusion/k_diffusion' '/content/gdrive/MyDrive/stable-diffusion-webui/scripts'
  !cp -R '/content/gdrive/MyDrive/stable-diffusion-webui/optimizedSD' '/content/gdrive/MyDrive/stable-diffusion-webui/scripts'
  !cp -R '/content/gdrive/MyDrive/stable-diffusion-webui/src/xformers/xformers' '/content/gdrive/MyDrive/stable-diffusion-webui/scripts'

s = getoutput('nvidia-smi')
if 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'

if (gpu=='T4'):
  %cd /content/
  !git clone https://github.com/TheLastBen/fast-stable-diffusion
  %cd /content/fast-stable-diffusion/precompiled
  !mv /content/fast-stable-diffusion/precompiled/_C_flashattention.1 /content/fast-stable-diffusion/precompiled/_C_flashattention.7z.001
  !mv /content/fast-stable-diffusion/precompiled/_C_flashattention.2 /content/fast-stable-diffusion/precompiled/_C_flashattention.7z.002
  !7z x /content/fast-stable-diffusion/precompiled/_C_flashattention.7z.001
  !mv -f /content/fast-stable-diffusion/precompiled/_C_flashattention.so /content/gdrive/MyDrive/stable-diffusion-webui/scripts/xformers
  !mv -f /content/fast-stable-diffusion/precompiled/_C.so /content/gdrive/MyDrive/stable-diffusion-webui/scripts/xformers


elif (gpu=='P100'):
  %cd /content/
  !git clone https://github.com/TheLastBen/fast-stable-diffusion
  %cd /content/fast-stable-diffusion/precompiled
  !mv /content/fast-stable-diffusion/precompiled/_C_flashattention-p100.1 /content/fast-stable-diffusion/precompiled/_C_flashattention.7z.001
  !mv /content/fast-stable-diffusion/precompiled/_C_flashattention-p100.2 /content/fast-stable-diffusion/precompiled/_C_flashattention.7z.002
  !7z x /content/fast-stable-diffusion/precompiled/_C_flashattention.7z.001
  !mv -f /content/fast-stable-diffusion/precompiled/_C.flashattention.so /content/gdrive/MyDrive/stable-diffusion-webui/scripts/xformers/_C_flashattention.so
  !mv -f /content/fast-stable-diffusion/precompiled/_C-p100.so /content/gdrive/MyDrive/stable-diffusion-webui/scripts/xformers/_C.so
  
%cd /content/gdrive/MyDrive/stable-diffusion-webui/scripts/ldm/modules


In [7]:
#@markdown # Patching attention.py
%%writefile attention.py
import gc
from inspect import isfunction
import math
import torch
import torch.nn.functional as F
from torch import nn, einsum
from einops import rearrange, repeat
import os
from typing import Any, Optional
import xformers
import xformers.ops

   

from ldm.modules.diffusionmodules.util import checkpoint


def exists(val):
    return val is not None


def uniq(arr):
    return{el: True for el in arr}.keys()


def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d


def max_neg_value(t):
    return -torch.finfo(t.dtype).max


def init_(tensor):
    dim = tensor.shape[-1]
    std = 1 / math.sqrt(dim)
    tensor.uniform_(-std, std)
    return tensor





# feedforward
class GEGLU(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.proj = nn.Linear(dim_in, dim_out * 2)

    def forward(self, x):
        x, gate = self.proj(x).chunk(2, dim=-1)
        return x * F.gelu(gate)


class FeedForward(nn.Module):
    def __init__(self, dim, dim_out=None, mult=4, glu=False, dropout=0.):
        super().__init__()
        inner_dim = int(dim * mult)
        dim_out = default(dim_out, dim)
        project_in = nn.Sequential(
            nn.Linear(dim, inner_dim),
            nn.GELU()
        ) if not glu else GEGLU(dim, inner_dim)

        self.net = nn.Sequential(
            project_in,
            nn.Dropout(dropout),
            nn.Linear(inner_dim, dim_out)
        )

    def forward(self, x):
        return self.net(x)


def zero_module(module):
    """
    Zero out the parameters of a module and return it.
    """
    for p in module.parameters():
        p.detach().zero_()
    return module


def Normalize(in_channels):
    return torch.nn.GroupNorm(num_groups=32, num_channels=in_channels, eps=1e-6, affine=True)


class LinearAttention(nn.Module):
    def __init__(self, dim, heads=4, dim_head=32):
        super().__init__()
        self.heads = heads
        hidden_dim = dim_head * heads
        self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias = False)
        self.to_out = nn.Conv2d(hidden_dim, dim, 1)

    def forward(self, x):
        b, c, h, w = x.shape
        qkv = self.to_qkv(x)
        q, k, v = rearrange(qkv, 'b (qkv heads c) h w -> qkv b heads c (h w)', heads = self.heads, qkv=3)
        k = k.softmax(dim=-1)
        context = torch.einsum('bhdn,bhen->bhde', k, v)
        out = torch.einsum('bhde,bhdn->bhen', context, q)
        out = rearrange(out, 'b heads c (h w) -> b (heads c) h w', heads=self.heads, h=h, w=w)
        return self.to_out(out)


class SpatialSelfAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = Normalize(in_channels)
        self.q = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.k = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.v = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.proj_out = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)

    def forward(self, x):
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b,c,h,w = q.shape
        q = rearrange(q, 'b c h w -> b (h w) c')
        k = rearrange(k, 'b c h w -> b c (h w)')
        w_ = torch.einsum('bij,bjk->bik', q, k)

        w_ = w_ * (int(c)**(-0.5))
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = rearrange(v, 'b c h w -> b c (h w)')
        w_ = rearrange(w_, 'b i j -> b j i')
        h_ = torch.einsum('bij,bjk->bik', v, w_)
        h_ = rearrange(h_, 'b c (h w) -> b c h w', h=h)
        h_ = self.proj_out(h_)

        return x+h_


class CrossAttention(nn.Module):
    def __init__(self, query_dim, context_dim=None, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        inner_dim = dim_head * heads
        context_dim = default(context_dim, query_dim)

        self.scale = dim_head ** -0.5
        self.heads = heads

        self.to_q = nn.Linear(query_dim, inner_dim, bias=False)
        self.to_k = nn.Linear(context_dim, inner_dim, bias=False)
        self.to_v = nn.Linear(context_dim, inner_dim, bias=False)
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, query_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x, context=None, mask=None):
        h = self.heads

        q_in = self.to_q(x)
        context = default(context, x)
        k_in = self.to_k(context)
        v_in = self.to_v(context)
        del context, x

        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h=h), (q_in, k_in, v_in))
        del q_in, k_in, v_in

        r1 = torch.zeros(q.shape[0], q.shape[1], v.shape[2], device=q.device)

        stats = torch.cuda.memory_stats(q.device)
        mem_active = stats['active_bytes.all.current']
        mem_reserved = stats['reserved_bytes.all.current']
        mem_free_cuda, _ = torch.cuda.mem_get_info(torch.cuda.current_device())
        mem_free_torch = mem_reserved - mem_active
        mem_free_total = mem_free_cuda + mem_free_torch

        gb = 1024 ** 3
        tensor_size = q.shape[0] * q.shape[1] * k.shape[1] * q.element_size()
        modifier = 3 if q.element_size() == 2 else 2.5
        mem_required = tensor_size * modifier
        steps = 1


        if mem_required > mem_free_total:
            steps = 2**(math.ceil(math.log(mem_required / mem_free_total, 2)))
            # print(f"Expected tensor size:{tensor_size/gb:0.1f}GB, cuda free:{mem_free_cuda/gb:0.1f}GB "
            #      f"torch free:{mem_free_torch/gb:0.1f} total:{mem_free_total/gb:0.1f} steps:{steps}")

        if steps > 64:
            max_res = math.floor(math.sqrt(math.sqrt(mem_free_total / 2.5)) / 8) * 64
            raise RuntimeError(f'Not enough memory, use lower resolution (max approx. {max_res}x{max_res}). '
                               f'Need: {mem_required/64/gb:0.1f}GB free, Have:{mem_free_total/gb:0.1f}GB free')

        slice_size = q.shape[1] // steps if (q.shape[1] % steps) == 0 else q.shape[1]
        for i in range(0, q.shape[1], slice_size):
            end = i + slice_size
            s1 = einsum('b i d, b j d -> b i j', q[:, i:end], k) * self.scale

            s2 = s1.softmax(dim=-1, dtype=q.dtype)
            del s1

            r1[:, i:end] = einsum('b i j, b j d -> b i d', s2, v)
            del s2

        del q, k, v

        r2 = rearrange(r1, '(b h) n d -> b n (h d)', h=h)
        del r1

        return self.to_out(r2)


class BasicTransformerBlock(nn.Module):
    def __init__(self, dim, n_heads, d_head, dropout=0., context_dim=None, gated_ff=True, checkpoint=True):
        super().__init__()
        AttentionBuilder = MemoryEfficientCrossAttention        
        self.attn1 = AttentionBuilder(query_dim=dim, heads=n_heads, dim_head=d_head, dropout=dropout)  # is a self-attention
        self.ff = FeedForward(dim, dropout=dropout, glu=gated_ff)
        self.attn2 = AttentionBuilder(query_dim=dim, context_dim=context_dim,
                                    heads=n_heads, dim_head=d_head, dropout=dropout)  # is self-attn if context is none
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.norm3 = nn.LayerNorm(dim)
        self.checkpoint = checkpoint
        
    def _set_attention_slice(self, slice_size):
        self.attn1._slice_size = slice_size
        self.attn2._slice_size = slice_size

    def forward(self, hidden_states, context=None):
        hidden_states = hidden_states.contiguous() if hidden_states.device.type == "mps" else hidden_states
        hidden_states = self.attn1(self.norm1(hidden_states)) + hidden_states
        hidden_states = self.attn2(self.norm2(hidden_states), context=context) + hidden_states
        hidden_states = self.ff(self.norm3(hidden_states)) + hidden_states
        return hidden_states        

    # def forward(self, x, context=None):
        # return checkpoint(self._forward, (x, context), self.parameters(), self.checkpoint)

    # def _forward(self, x, context=None):
        # x = self.attn1(self.norm1(x)) + x
        # x = self.attn2(self.norm2(x), context=context) + x
        # x = self.ff(self.norm3(x)) + x
        # return x

class MemoryEfficientCrossAttention(nn.Module):
    def __init__(self, query_dim, context_dim=None, heads=8, dim_head=64, dropout=0.0):
        super().__init__()
        inner_dim = dim_head * heads
        context_dim = default(context_dim, query_dim)

        self.scale = dim_head**-0.5
        self.heads = heads
        self.dim_head = dim_head

        self.to_q = nn.Linear(query_dim, inner_dim, bias=False)
        self.to_k = nn.Linear(context_dim, inner_dim, bias=False)
        self.to_v = nn.Linear(context_dim, inner_dim, bias=False)

        self.to_out = nn.Sequential(nn.Linear(inner_dim, query_dim), nn.Dropout(dropout))
        self.attention_op: Optional[Any] = None

    def _maybe_init(self, x):
        """
        Initialize the attention operator, if required We expect the head dimension to be exposed here, meaning that x
        : B, Head, Length
        """
        if self.attention_op is not None:
            return

        _, M, K = x.shape
        try:
            self.attention_op = xformers.ops.AttentionOpDispatch(
                dtype=x.dtype,
                device=x.device,
                k=K,
                attn_bias_type=type(None),
                has_dropout=False,
                kv_len=M,
                q_len=M,
            ).op

        except NotImplementedError as err:
            raise NotImplementedError(f"Please install xformers with the flash attention / cutlass components.\n{err}")

    def forward(self, x, context=None, mask=None):


        q = self.to_q(x)
        context = default(context, x)
        k = self.to_k(context)
        v = self.to_v(context)
        


        b, _, _ = q.shape
        q, k, v = map(
            lambda t: t.unsqueeze(3)
            .reshape(b, t.shape[1], self.heads, self.dim_head)
            .permute(0, 2, 1, 3)
            .reshape(b * self.heads, t.shape[1], self.dim_head)
            .contiguous(),
            (q, k, v),
        )

        # init the attention op, if required, using the proper dimensions
        self._maybe_init(q)

        # actually compute the attention, what we cannot get enough of
        out = xformers.ops.memory_efficient_attention(q, k, v, attn_bias=None, op=self.attention_op)

        # TODO: Use this directly in the attention operation, as a bias
        if exists(mask):
            raise NotImplementedError
        out = (
            out.unsqueeze(0)
            .reshape(b, self.heads, out.shape[1], self.dim_head)
            .permute(0, 2, 1, 3)
            .reshape(b, out.shape[1], self.heads * self.dim_head)
        )

        stats = torch.cuda.memory_stats(q.device)
        mem_active = stats['active_bytes.all.current']
        mem_reserved = stats['reserved_bytes.all.current']
        mem_free_cuda, _ = torch.cuda.mem_get_info(torch.cuda.current_device())
        mem_free_torch = mem_reserved - mem_active
        mem_free_total = mem_free_cuda + mem_free_torch

        gb = 1024 ** 3
        tensor_size = q.shape[0] * q.shape[1] * k.shape[1] * q.element_size()
        modifier = 3 if q.element_size() == 2 else 2.5
        mem_required = tensor_size * modifier
        steps = 1


        if mem_required > mem_free_total:
            steps = 2**(math.ceil(math.log(mem_required / mem_free_total, 2)))
            # print(f"Expected tensor size:{tensor_size/gb:0.1f}GB, cuda free:{mem_free_cuda/gb:0.1f}GB "
            #      f"torch free:{mem_free_torch/gb:0.1f} total:{mem_free_total/gb:0.1f} steps:{steps}")

        if steps > 64:
            max_res = math.floor(math.sqrt(math.sqrt(mem_free_total / 2.5)) / 8) * 64
            raise RuntimeError(f'Not enough memory, use lower resolution (max approx. {max_res}x{max_res}). '
                               f'Need: {mem_required/64/gb:0.1f}GB free, Have:{mem_free_total/gb:0.1f}GB free')


        return self.to_out(out)




class SpatialTransformer(nn.Module):
    """
    Transformer block for image-like data.
    First, project the input (aka embedding)
    and reshape to b, t, d.
    Then apply standard transformer action.
    Finally, reshape to image
    """
    def __init__(self, in_channels, n_heads, d_head,
                 depth=1, dropout=0., context_dim=None):
        super().__init__()
        self.in_channels = in_channels
        inner_dim = n_heads * d_head
        self.norm = Normalize(in_channels)

        self.proj_in = nn.Conv2d(in_channels,
                                 inner_dim,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)

        self.transformer_blocks = nn.ModuleList(
            [BasicTransformerBlock(inner_dim, n_heads, d_head, dropout=dropout, context_dim=context_dim)
                for d in range(depth)]
        )

        self.proj_out = zero_module(nn.Conv2d(inner_dim,
                                              in_channels,
                                              kernel_size=1,
                                              stride=1,
                                              padding=0))

    def forward(self, x, context=None):
        # note: if no context is given, cross-attention defaults to self-attention
        b, c, h, w = x.shape
        x_in = x
        x = self.norm(x)
        x = self.proj_in(x)
        x = rearrange(x, 'b c h w -> b (h w) c')
        for block in self.transformer_blocks:
            x = block(x, context=context)
        x = rearrange(x, 'b (h w) c -> b c h w', h=h, w=w)
        x = self.proj_out(x)
        return x + x_in

Overwriting attention.py


In [ ]:
#@markdown # Start Stable Diffusion
!mv /content/u1000.ckpt /content/gdrive/MyDrive/stable-diffusion-webui/models/ldm/stable-diffusion-v1/model.ckpt
!ls -la /content/gdrive/MyDrive/stable-diffusion-webui/models/ldm/stable-diffusion-v1/model.ckpt
%cd /content/gdrive/MyDrive/stable-diffusion-webui/
!wget https://pomf2.lain.la/f/bz46z5a5.ttf -O arial.ttf
!python /content/gdrive/MyDrive/stable-diffusion-webui/scripts/webui.py --share

mv: cannot stat '/content/u1000.ckpt': No such file or directory
-rw-------+ 1 root root 2132887649 Sep 28 08:34 /content/gdrive/MyDrive/stable-diffusion-webui/models/ldm/stable-diffusion-v1/model.ckpt
/content/gdrive/MyDrive/stable-diffusion-webui
--2022-09-28 09:03:55--  https://pomf2.lain.la/f/bz46z5a5.ttf
Resolving pomf2.lain.la (pomf2.lain.la)... 198.251.81.242, 198.251.81.32, 198.251.82.91, ...
Connecting to pomf2.lain.la (pomf2.lain.la)|198.251.81.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275572 (269K) [application/octet-stream]
Saving to: ‘arial.ttf’

arial.ttf           100%[===================>] 269.11K  --.-KB/s    in 0.1s    

2022-09-28 09:03:55 (2.66 MB/s) - ‘arial.ttf’ saved [275572/275572]

Found GFPGAN
Found RealESRGAN
Found LDSR
Loading model from models/ldm/stable-diffusion-v1/model.ckpt
Global Step: 1000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 51